## Import libraries

In [1]:
import src.config as config

2025-06-22 18:28:18.574 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Repositories\ds-lstm-ibov


## ETL — Extract, Transform, Load of Raw Dataset

This section is responsible for the Extraction phase of the ETL process, pulling historical financial data from multiple sources as defined in the configuration file [dataset.yaml](../configs/dataset.yaml).

- Data Storage: Each dataset is stored locally in the data/raw/ directory, organized and saved individually as .csv files for traceability and versioning.
- Data Cleaning:
  - Missing values are treated using a standard cleaning strategy.
  - Features with low variance (threshold < 0.01) are removed to reduce noise and improve modeling efficiency. In this execution, no low-variance columns were found.

In [2]:
from src import dataset

dataset.main(
    # ---- REPLACE DEFAULT AS APPROPRIATE ----
    asset = '^BVSP',
    asset_focus = 'Close',
    years = 5
    # -----------------------------------------
    )

2025-06-22 18:28:21.364 | INFO     | src.dataset:main:32 - Starting raw data loading...
2025-06-22 18:28:21.364 | INFO     | src.dataset:main:38 - Requesting information between 2020-06-23 and 2025-06-22
2025-06-22 18:28:21.364 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BOVESPA (^BVSP) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-22 18:28:24.545 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading S&P500 (^GSPC) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-22 18:28:27.177 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BITCOIN (BTC-USD) from yfinance...


...

2025-06-22 18:28:35.088 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading ACUCAR (SB=F) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-22 18:28:37.704 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC (11) from the Central Bank of Brazil API...
2025-06-22 18:28:45.500 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading CDI (12) from the Central Bank of Brazil API...
2025-06-22 18:28:53.194 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Anual (1178) from the Central Bank of Brazil API...
2025-06-22 18:29:00.920 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Meta_Anual (432) from the Central Bank of Brazil API...
2025-06-22 18:29:13.903 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading IPCA_Mensal (433) from the Central Bank of Brazil API...
2025-06-22 18:29:16.624 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading IGP_M_Mensal (189) from the Central Bank of Brazil API...
2025-06-22 18:29:19.082 | INFO     | src.utils.dataset.

INFO:src.utils.dataset.clean_strategy:Executing CleanMissingValues...
INFO:src.utils.dataset.clean_strategy:Executing CleanLowVariance with threshold=0.01...
INFO:src.utils.dataset.clean_strategy:Columns removed due to low variance: Index([], dtype='object')


2025-06-22 18:29:47.916 | SUCCESS  | src.dataset:main:64 - Raw data successfully loaded...
            ('Close', '^BVSP')  ('High', '^BVSP')  ('Low', '^BVSP')  \
2025-06-16              2043.0             2188.0             626.0   
2025-06-17              -416.0             -491.0            1081.0   
2025-06-18              -123.0             -336.0             150.0   
2025-06-19              -123.0             -336.0             150.0   
2025-06-20             -1601.0             -442.0           -1628.0   
2025-06-21             -1601.0             -442.0           -1628.0   
2025-06-22             -1601.0             -442.0           -1628.0   

            ('Open', '^BVSP')  ('Volume', '^BVSP')  ('Close', '^GSPC')  \
2025-06-16             -588.0           -1007800.0           56.139648   
2025-06-17             2044.0             756500.0          -50.389648   
2025-06-18             -412.0             -53600.0           -1.850098   
2025-06-19             -412.0             -5

## Feature Engineering — Time Series Preparation

This part of the pipeline is responsible for transforming the cleaned dataset into structured features suitable for training time series models. It includes the following key steps:
- Feature Generation: Constructs relevant features based on historical market data.
- Dataset Splitting: The dataset is split into training and testing sets using a consistent strategy to preserve temporal structure.
- Time Series Windowing: Converts the sequential data into overlapping windows, enabling the model to learn temporal dependencies.
- Saving Artifacts: Both training and testing sets are stored for reproducibility, along with the transformation pipelines applied during preprocessing.

In [3]:
from src import features

features.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    train_dir = config.PROCESSED_DATA_DIR,
    test_dir = config.PROCESSED_DATA_DIR,
    # -----------------------------------------
)

2025-06-22 18:29:52.930 | INFO     | src.features:main:31 - Generating features from dataset...
2025-06-22 18:29:52.981 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-22 18:29:53.014 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...
2025-06-22 18:29:53.481 | SUCCESS  | src.features:main:46 - Saving train features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-22 18:29:54.131 | SUCCESS  | src.features:main:50 - Saving test features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-22 18:29:54.131 | SUCCESS  | src.features:main:54 - Features generation complete.
2025-06-22 18:29:54.131 | INFO     | src.features:main:56 - Saving transformers...
2025-06-22 18:29:54.131 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-22 18:29:54.131 | INFO     | src.features:main:72 - Total t

## Modeling

The modeling process begins with loading the training dataset. Next, the base model is constructed, and both the compilation and training strategies are defined. With the pipeline structure in place, the model is trained over 100 epochs using an iterative approach to adjust the weights.

During training, key metrics are monitored, including accuracy, loss (error), validation accuracy and loss (on unseen data), as well as the learning rate.

In [4]:
from src.modeling import train

train.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    X_path = config.PROCESSED_DATA_DIR / "X_train.npy",
    y_path = config.PROCESSED_DATA_DIR / "y_train.npy"
    # -----------------------------------------
)

2025-06-22 18:29:54.283 | INFO     | src.modeling.train:main:29 - Loading training dataset...
2025-06-22 18:29:54.573 | INFO     | src.modeling.train:main:37 - Building model...
2025-06-22 18:29:54.574 | INFO     | src.modeling.train:main:43 - Selecting compile strategy...
2025-06-22 18:29:54.574 | INFO     | src.modeling.train:main:46 - Selecting training strategy...
2025-06-22 18:29:54.574 | INFO     | src.modeling.train:main:49 - Building model training pipeline template...
2025-06-22 18:29:54.574 | INFO     | src.modeling.train:main:56 - Training model...
Epoch 1/1000
222/222 ━━━━━━━━━━━━━━━━━━━━ 77s 309ms/step - loss: 0.3709 - mae: 0.6029 - mse: 0.7600 - r2_score: -0.3129 - rmse: 0.7827 - smape: 166.1439 - val_loss: 0.4005 - val_mae: 0.6921 - val_mse: 0.8220 - val_r2_score: -0.1151 - val_rmse: 0.9066 - val_smape: 161.0964 - learning_rate: 0.0100
Epoch 2/1000
222/222 ━━━━━━━━━━━━━━━━━━━━ 67s 303ms/step - loss: 0.2819 - mae: 0.5501 - mse: 0.6784 - r2_score: -0.0015 - rmse: 0.7277 - 

KeyboardInterrupt: 

## Predict

In the prediction stage, the trained model is loaded along with the input data. The model then performs inference, generating predictions based on the provided data.


In [ ]:
from src.modeling import predict
predict.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    preprocessor_path = config.PROCESSED_DATA_DIR / "preprocessor.pkl",
    model_path = config.MODELS_DIR / "Sequential_epoch100_loss0.1747.keras",  # Select the best model  
    postprocessor_path = config.PROCESSED_DATA_DIR / "postprocessor.pkl",
    output_path = config.PROCESSED_DATA_DIR / "dataset_report.csv",
    # -----------------------------------------
)

## Plot

Open the [Power BI Report]((../reports/pbi/amp-fynance.pbip)) and refresh data.